<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## Lab: Cleaning Rock Song Data

_Authors: Dave Yerrington (SF)_

---


**JACOB"S SOLUTIONS**

In [86]:
import pandas as pd
import numpy as np 
import seaborn as sns

%matplotlib inline

### 1. Load `rock.csv` and do an initial examination of its data columns.

In [87]:
rockfile = "./datasets/rock.csv"

In [88]:
# Load the data.

rock = pd.read_csv(rockfile)

In [89]:
rock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 8 columns):
Song Clean      2230 non-null object
ARTIST CLEAN    2230 non-null object
Release Year    1653 non-null object
COMBINED        2230 non-null object
First?          2230 non-null int64
Year?           2230 non-null int64
PlayCount       2230 non-null int64
F*G             2230 non-null int64
dtypes: int64(4), object(4)
memory usage: 139.5+ KB


In [90]:
# Look at the information regarding its columns.

### 2.  Clean up the column names.

Let's clean up the column names. There are two ways we can accomplish this:

#### 2.A Change the column names when you import the data using `pd.read_csv()`.

Notice that, when passing `names=[..A LIST OF STRING..]` with a number of columns that matches the number of strings in the passed list, you replace the column names.

NOTE: When you create custom column names, the first row of the `.csv` already represents a header. It is important to tell `pandas` to skip that row. The `skiprows=1` keyword argument to `read_csv()` will tell `pandas` to skip the first row.

In [91]:
# Change the column names when loading the '.csv':
col_names = ['song_clean', 'artist_clean', 'release_year', 'combined', 'first', 'year', 'playcount', 'fg']
rock2 = pd.read_csv(rockfile, names = col_names, skiprows= 1)

In [92]:
rock2.head()

,song_clean,artist_clean,release_year,combined,first,year,playcount,fg
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


#### 2.B Change column names using the `.rename()` function.

The `.rename()` function takes an argument, `columns=name_dict`, in which `name_dict` is a dictionary containing the original column names as keys and the new column names as values.

In [93]:
# Change the column names using the `.rename()` function.
rock.rename(columns = {'Song Clean': 'song_clean', 'Release Year': 'release', 'COMBINED': 'combo'}, inplace = True)

In [94]:
rock.head()

,song_clean,ARTIST CLEAN,release,combo,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


#### 2.C Reassigning the `.columns` attribute of a DataFrame.

You can also just reassign the `.columns` attribute to a list of strings containing the new column names. 

The only caveat with reassigning `.columns` is that you have to reassign all of the column names at once. You can't partially replace a value by working on `.columns` directly. You have to reassign `.columns` with a list of equal length. 

In [95]:
# Replace the column names by reassigning the `.columns` attribute.
#rock.columns = ['song_clean', etc.]

### 3. Subsetting data where null values exist.

We have mixed `str` and `NaN` values in the `release` column. `NaN` stands for "not a number" and is the way `pandas` handles "nulls" or nonexistent data. We can use the `.isnull()` method of a Series to find null values.

Print the header of the data subset to where the `release` column is null values.

In [96]:
# Show records where df['release'] is null
rock[rock['release'].isnull()].head()

,song_clean,ARTIST CLEAN,release,combo,First?,Year?,PlayCount,F*G
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
10,"Baby, Please Don't Go",AC/DC,NaN,"Baby, Please Don't Go by AC/DC",1,0,1,0
13,CAN'T STOP ROCK'N'ROLL,AC/DC,NaN,CAN'T STOP ROCK'N'ROLL by AC/DC,1,0,5,0
16,Girls Got Rhythm,AC/DC,NaN,Girls Got Rhythm by AC/DC,1,0,24,0
24,Let's Get It Up,AC/DC,NaN,Let's Get It Up by AC/DC,1,0,4,0


### 4. Update slices of your DataFrame based on mask selection/slices.

In many scenarios, we want to upate values in our DataFrame according to criteria. Let's say we wanted to set all of the null values in `release` to 0.

With newer versions of `pandas`, in order to manipulate data in the original DataFrame, we have to use `.loc` while performing reassignment using a mask and an index.

For example, the following won't always work:
```python
df[row_mask]['column_name'] = new_value
```

The best way to accomplish the same task is:
```python
df.loc[row_mask, 'column_name'] = new_value
```

For multiple column assignment, you would use:
```python
df.loc[row_mask, ['col_1', 'col_2', 'col_3']] = new_value
```

#### 4.A Let's try it out. Make all of the null values in `release` 0.

In [97]:
# Replace release nulls with 0
rock.loc[rock['release'].isnull(), ['release']] = 0

In [98]:
rock.release.value_counts(dropna= False).head()

0       577
1973    104
1977     83
1975     83
1970     81
Name: release, dtype: int64

#### 4.B Verify that `release` contains no null values.

In [99]:
# A:
rock['release'].isnull().sum()

0

### 5. Ensure that the data types of the columns make sense. 

Verifying column data types is a critical part of data munging. If columns have the wrong data type, then there is usually corrupted or incorrect data in some of the observations.

#### 5.A Look at the data types for the columns. Are any incorrect given what the data represents?

In [100]:
# A:
rock.dtypes

song_clean      object
ARTIST CLEAN    object
release         object
combo           object
First?           int64
Year?            int64
PlayCount        int64
F*G              int64
dtype: object

### 6. Investigate and clean up the `release` column.

The `release` column is a string data type when it should be an integer.

#### 6.A Figure out what value(s) are causing the `release` column to be encoded as a string instead of an integer.

In [101]:
# A:
rock.release.astype('int')

ValueError: invalid literal for int() with base 10: 'SONGFACTS.COM'

#### 6.B Look at the rows in which there is incorrect data in the `release` column.

In [102]:
# A:
rock[rock['release'] == 'SONGFACTS.COM']

,song_clean,ARTIST CLEAN,release,combo,First?,Year?,PlayCount,F*G
1504,Bullfrog Blues,Rory Gallagher,SONGFACTS.COM,Bullfrog Blues by Rory Gallagher,1,1,1,1


#### 6.C. Clean up the data. Normally we may replace the offending data with null np.nan values, however we previously converted all of the nan values in the release column to zeros so we might as well continue with the same practice. Replacing with 0 (or nan) will allow us to convert the column to numeric.

In [103]:
# A:
rock.loc[1504, 'release'] = 0

In [104]:
rock['release'] = rock['release'].astype('int')

### 7. Get summary statistics for the `release` column using the `.describe()` function.

Now that the `release` column is finally a numeric data type, we can apply the `.describe()` function.  

#### 7.A Print out the summary stats for the `release` column. What is the earliest and latest release date?

In [105]:
# A:
rock['release'].describe()

count    2230.000000
mean     1465.331390
std       867.196161
min         0.000000
25%         0.000000
50%      1973.000000
75%      1981.000000
max      2014.000000
Name: release, dtype: float64

#### 7.B Based on the summary statistics, is there anything else wrong with the `release` column? 

In [106]:
# A: Seems there are an awful lot of zeros... a quarter of the data!

_Looking at the DataFrame that contains the year 1071, we can see that the year was probably corrupted and should be replaced with something else if possible._

### 8. Make changes and investigate using custom functions with `.apply()`.

Let's say we want to traverse every single row in our data set and apply a function to that row.

#### 8.A Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970.


In [107]:
# A:
def seventies_test(row):
    print(rock.loc[row, 'song_clean'])
    print(rock.loc[row, 'ARTIST CLEAN'])
    if rock.loc[row, 'release'] < 1970:
        print('True')
    else:
        print('False')

#### 8.B Using the `.apply()` function, apply the function you wrote to the first four rows of the DataFrame.

You will need to tell the `apply` function to operate row by row. Setting the keyword argument as `axis=1` indicates that the function should be applied to each row individually.

In [108]:
# A:
rock['release'][:4].apply(seventies_test)

Blitzkrieg Bop
The Ramones
False
Caught Up in You
.38 Special
False
TALK OF THE TOWN
The Pretenders
False
Mystery Achievement
The Pretenders
True


0    None
1    None
2    None
3    None
Name: release, dtype: object

In [109]:
def seventies_better(row):
    song = rock.loc[row, 'song_clean']
    artist = rock.loc[row, 'ARTIST CLEAN']
    if rock.loc[row, 'release'] < 1970:
        sevs = 'True'
    else:
        sevs = 'False'
    print('The song is {} the artist is {} and it would be {} to say that the album was released before 1970.'.format(song, artist, sevs))

In [110]:
rock['release'][:4].apply(seventies_better)

The song is Blitzkrieg Bop the artist is The Ramones and it would be False to say that the album was released before 1970.
The song is Caught Up in You the artist is .38 Special and it would be False to say that the album was released before 1970.
The song is TALK OF THE TOWN the artist is The Pretenders and it would be False to say that the album was released before 1970.
The song is Mystery Achievement the artist is The Pretenders and it would be True to say that the album was released before 1970.


0    None
1    None
2    None
3    None
Name: release, dtype: object

You'll notice that there will be a final output Series of `None` values. The `.apply()` function, if a return value is not specified, will return a Series of `None` values (similar to how the default return for Python functions is `None` when a return statement is not specified).

### 9. Write a function that converts cells in a DataFrame to float and otherwise replaces them with `np.nan`.

If applied to our data, it would keep only the numeric information and otherwise input null values.

Recall that the try-except syntax in Python is a great way to try something and take another action if the initial step fails:

```python
try:
    Perform some action.
except:
   Perform some other action if the first failed with an error.
```

#### 9.A Write the function that takes a column and converts all of its values to float if possible and `np.nan` otherwise. The return value should be the converted Series.

In [111]:
# A:
def checker(value):
    try:
        return float(value)
    except:
        return np.nan

In [112]:
checker(5)

5.0

In [113]:
checker('stever')

nan

In [120]:
def changer(col):
    col = col.map(checker)
    return col    

#### 9.B Try your function out on the rock song data and ensure the output is what you expected.


In [115]:
# A:
rock.apply(changer).head()

#### 9.C Describe the new float-only DataFrame.

In [123]:
# A:
df2 = rock.apply(changer)
df2.describe()

,song_clean,ARTIST CLEAN,release,combo,First?,Year?,PlayCount,F*G
count,2.000000,0.0,2230.000000,0.0,2230.0,2230.000000,2230.000000,2230.000000
mean,1012.000000,NaN,1465.331390,NaN,1.0,0.741256,16.872646,15.048430
std,1367.544515,NaN,867.196161,NaN,0.0,0.438043,25.302972,25.288366
min,45.000000,NaN,0.000000,NaN,1.0,0.000000,0.000000,0.000000
25%,528.500000,NaN,0.000000,NaN,1.0,0.000000,1.000000,0.000000
50%,1012.000000,NaN,1973.000000,NaN,1.0,1.000000,4.000000,3.000000
75%,1495.500000,NaN,1981.000000,NaN,1.0,1.000000,21.000000,18.000000
max,1979.000000,NaN,2014.000000,NaN,1.0,1.000000,142.000000,142.000000
